In [8]:
import pandas as pd
import numpy as np

In [10]:
temp_df_pt1 = pd.read_csv('NOAAGlobalTemp_v5a_1.csv')
temp_df_pt2 = pd.read_csv('NOAAGlobalTemp_v5a_2.csv')

# concatenate dataframes
# gonna turn all the wrong years into -1
frames = [temp_df_pt1, temp_df_pt2]
temp_df = pd.concat(frames, sort=False)
temp_df

tree_df = pd.read_csv('tree_data.csv')
tree_df['year'] = tree_df['year'].apply(lambda x: float(x) if str(x).isdigit() else -1)
tree_df.head()

,year,avg_tree_ring_width,lat,long,elevation,tree_species,organism_group
0,969.0,164.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
1,970.0,188.5,51.75041,-0.34223,NaN,Quercus spp. L.,0
2,971.0,139.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
3,972.0,173.0,51.75041,-0.34223,NaN,Quercus spp. L.,0
4,973.0,221.0,51.75041,-0.34223,NaN,Quercus spp. L.,0


In [12]:
# How many have bad years
tree_df[tree_df['year'] == -1].shape

(120637, 7)

In [13]:
def get_5x5_grid(og_lat, og_lon, base=2.5):
    lat1 = base * round((og_lat-2.5)/base)
    lat2 = base * round((og_lat+2.5)/base)

    lon1 = base * round((og_lon-2.5)/base)
    lon2 = base * round((og_lon+2.5)/base)

    if lat2-2.5 < og_lat:
        lat2 += 2.5
        lat1 += 2.5
    else:
        lat2 -= 2.5
        lat1 -= 2.5

    if lon2-2.5 < og_lon:
        lon2 += 2.5
        lon1 += 2.5
    else:
        lon2 -= 2.5
        lon1 -= 2.5

    if (lat1 % 5 )== 0 or (lat2 % 5 )== 0:
        if lat1 < 0:
            lat1 += 2.5
            lat2 += 2.5
        else: 
            lat1 -= 2.5
            lat2 -= 2.5

    if (lon1 % 5 )== 0 or (lon2 % 5 )== 0:
        lon1 += 2.5
        lon2 += 2.5

    if (lat1 == -92.5 and lat2 == -87.5):
        lat1 = -87.5
        lat2 = -82.5
    elif (lat1 == 87.5 and lat2 == 92.5):
        lat1 = 82.5
        lat2 = 87.5
    
    if (lon1 == 0.0 and lon2 == 5.0):
        lon1 = 2.5
        lon2 = 7.5
    elif (lon1 == 355.0 and lon2 == 360.0):
        lon1 = 352.5
        lon2 = 357.5

    return lat1, lat2, lon1, lon2

In [14]:
# given an observation row, return the row with the correct 5x5 temperature anomaly

def get_gridded_temp_anom(lat, lon, year):
    lat1, lat2, lon1, lon2 = get_5x5_grid(lat, lon)
    temp_df_for_lat_lon = temp_df[((temp_df['lat'] == lat1) | (temp_df['lat'] == lat2)) &
                                  ((temp_df['lon'] == lon1) | (temp_df['lon'] == lon2)) &
                                  (temp_df['year'] == str(year))]
    annual_temp_df_for_lat_lon = temp_df_for_lat_lon.groupby(['year']).agg({'anom': ['mean']}).reset_index()
    if len(annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()) > 0:
        temp_anom = annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()[0]
        return temp_anom

In [15]:
temp_df['year'] = temp_df['time'].apply(lambda x: str(x)[0:4])
temp_df = temp_df.dropna(subset=['anom'])
temp_annual_df =  temp_df.groupby(['year', 'lat', 'lon']).agg({'anom': ['mean']}).reset_index()

0          1880
1          1880
2          1880
3          1880
4          1880
5          1880
6          1880
7          1880
8          1880
9          1880
10         1880
11         1880
12         1880
13         1880
14         1880
15         1880
16         1880
17         1880
18         1880
19         1880
20         1880
21         1880
22         1880
23         1880
24         1880
25         1880
26         1880
27         1880
28         1880
29         1880
           ... 
1307392    2021
1307393    2021
1307394    2021
1307395    2021
1307396    2021
1307397    2021
1307398    2021
1307399    2021
1307400    2021
1307401    2021
1307402    2021
1307403    2021
1307404    2021
1307405    2021
1307406    2021
1307407    2021
1307408    2021
1307409    2021
1307410    2021
1307411    2021
1307412    2021
1307413    2021
1307414    2021
1307415    2021
1307416    2021
1307417    2021
1307418    2021
1307419    2021
1307420    2021
1307421    2021
Name: year, Length: 4408

In [ ]:
# given an observation row, return the row with the correct 5x5 temperature anomaly

def get_gridded_temp_anom(lat, lon, year):
    lat1, lat2, lon1, lon2 = get_5x5_grid(lat, lon)
    temp_df_for_lat_lon = temp_df[((temp_df['lat'] == lat1) | (temp_df['lat'] == lat2)) &
                                  ((temp_df['lon'] == lon1) | (temp_df['lon'] == lon2)) &
                                  (temp_df['year'] == str(year))]
    annual_temp_df_for_lat_lon = temp_df_for_lat_lon.groupby(['year']).agg({'anom': ['mean']}).reset_index()
    if len(annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()) > 0:
        temp_anom = annual_temp_df_for_lat_lon[annual_temp_df_for_lat_lon['year']==str(year)]['anom']['mean'].tolist()[0]
        return temp_anom